In [1]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import AmlCompute
from azure.ai.ml import Input,Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.ai.ml.entities import Environment
from dotenv import dotenv_values
from dotenv import load_dotenv
# from src.login import get_ws_client

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
import os
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize DefaultAzureCredential
credential = DefaultAzureCredential()

# Load Key Vault details
keyVaultName = "snow755wss0305478416"
if not keyVaultName:
    logger.error("Key Vault name is not set in environment variables.")
    raise ValueError("Key Vault name is missing.")

key_vault_url = f"https://{keyVaultName}.vault.azure.net"

# Initialize the SecretClient
try:
    secret_client = SecretClient(vault_url=key_vault_url, credential=credential)
    logger.info("Connected to Azure Key Vault.")
except Exception as ex:
    logger.error(f"Error connecting to Azure Key Vault: {ex}")
    raise

# Retrieve secrets with caching
secrets_cache = {}

def get_secret(secret_name):
    if secret_name in secrets_cache:
        return secrets_cache[secret_name]
    secret_value = secret_client.get_secret(secret_name).value
    secrets_cache[secret_name] = secret_value
    return secret_value

try:
    subscription_id = get_secret("subscriptionid")
    resource_group = get_secret("resource-group")
    workspace = get_secret("workspace-name")
    logger.info("Secrets retrieved successfully.")
except Exception as ex:
    logger.error(f"Error retrieving secrets from Azure Key Vault: {ex}")
    raise

# Initialize MLClient
try:
    ml_client = MLClient(
        credential=credential,
        subscription_id=subscription_id,
        resource_group_name=resource_group,
        workspace_name=workspace,
    )
    logger.info("MLClient initialized successfully.")
except Exception as ex:
    logger.error(f"Error initializing MLClient: {ex}")
    raise




data_asset = ml_client.data.get(name="credit_card_datagyygg6", version="4")

INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:__main__:Connected to Azure Key Vault.
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://snow755wss0305478416.vault.azure.net/secrets/subscriptionid/?api-version=REDACTED'
Request method: 'GET'
Request headers:
    'Accept': 'application/json'
    'x-ms-client-request-id': 'cacf884f-aba3-11ef-bad3-2811a8cad455'
    'User-Agent': 'azsdk-python-keyvault-secrets/4.9.0 Python/3.10.11 (Windows-10-10.0.22631-SP0)'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 401
Response headers:
    'Cache-Control': 'no-cache'
    'Pragma': 'no-cache'
    'Content-Type': 'application/json; charset=utf-8'
    'Expires': '-1'
    'x-ms-keyvault-region': 'centralindia'
    'x-ms-client-request-id': 'cacf884f-aba3-11ef-bad3-2811a8cad455'
    'x-ms-requ

In [2]:
import pandas as pd
v1df = pd.read_csv(data_asset.path)

INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.identity._credentials.chained:DefaultAzureCredential acquired a token from AzureCliCredential
INFO:azure.identity._internal.decorators:AzureCliCredential.get_token_info succeeded
INFO:azure.identity._credentials.default:DefaultAzureCredential acquired a token from AzureCliCredential
INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.identity._credentials.chained:DefaultAzureCredential acquired a token from AzureCliCredential
INFO:azure.identity._internal.decorators:AzureCliCredential.get_token_info succeeded
INFO:azure.identity._credentials.default:DefaultAzureCredential acquired a token from AzureCliCredential


In [6]:
# read in data again, this time using the 2nd row as the header
df = pd.read_csv(data_asset.path, header=1)
# rename column
df.rename(columns={"default payment next month": "default"}, inplace=True)
# remove ID column
df.drop("ID", axis=1, inplace=True)

In [7]:
df.to_parquet("./data/cleaned-credit-card.parquet")

In [8]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential
import time


# Create version string for data asset
v2 = "cleaned" 
# + time.strftime("%Y.%m.%d.%H%M%S", time.gmtime())
my_path = "./data/cleaned-credit-card.parquet"

# Define the data asset with tags to make it clear the asset can be used in training
my_data = Data(
    name="credit-card-data",
    version=v2,
    description="Default of credit card clients data.",
    # tags={"training_data": "true", "format": "parquet"},
    path=my_path,
    type=AssetTypes.URI_FILE,
)

## create data asset if it doesn't already exist:
try:
    data_asset = ml_client.data.get(name="credit-card-data", version=v2)
    print(
        f"Data asset already exists. Name: {my_data.name}, version: {my_data.version}"
    )
except:
    ml_client.data.create_or_update(my_data)
    print(f"Data asset created. Name: {my_data.name}, version: {my_data.version}")


INFO:azure.identity._internal.decorators:AzureCliCredential.get_token_info succeeded
INFO:azure.identity._credentials.default:DefaultAzureCredential acquired a token from AzureCliCredential
Uploading cleaned-credit-card.parquet (< 1 MB): 100%|##########| 1.58M/1.58M [00:00<00:00, 1.64MB/s]




Data asset created. Name: credit-card-data, version: cleaned


In [16]:
print(data_asset.path) 

input_data = Input(path=data_asset.path) 
print(input_data)

azureml://subscriptions/cbea065c-379a-4b0b-afd2-e62026886cdf/resourcegroups/SNOW_755_RGG/workspaces/SNOW_755_WSS/datastores/workspaceblobstore/paths/LocalUpload/4d9270ba0cd3cd3ce724f5ef16c7b70b/cleaned-credit-card.parquet
{'type': 'uri_folder', 'path': 'azureml://subscriptions/cbea065c-379a-4b0b-afd2-e62026886cdf/resourcegroups/SNOW_755_RGG/workspaces/SNOW_755_WSS/datastores/workspaceblobstore/paths/LocalUpload/4d9270ba0cd3cd3ce724f5ef16c7b70b/cleaned-credit-card.parquet'}


In [28]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.compute import ComputeManagementClient

# Set up your Azure credentials
credential = DefaultAzureCredential()
compute_client = ComputeManagementClient(credential, subscription_id)

# List available VM sizes in a given region
region = "centralindia"
vm_sizes = compute_client.virtual_machine_sizes.list(location=region)

# Print the VM sizes
for vm in vm_sizes:
    print(f"VM Size: {vm.name}, Cores: {vm.number_of_cores}, Memory: {vm.memory_in_mb}MB")


INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.identity._credentials.chained:DefaultAzureCredential acquired a token from AzureCliCredential


VM Size: Standard_F2s_v2, Cores: 2, Memory: 4096MB
VM Size: Standard_F4s_v2, Cores: 4, Memory: 8192MB
VM Size: Standard_F8s_v2, Cores: 8, Memory: 16384MB
VM Size: Standard_F16s_v2, Cores: 16, Memory: 32768MB
VM Size: Standard_F32s_v2, Cores: 32, Memory: 65536MB
VM Size: Standard_F48s_v2, Cores: 48, Memory: 98304MB
VM Size: Standard_F64s_v2, Cores: 64, Memory: 131072MB
VM Size: Standard_F72s_v2, Cores: 72, Memory: 147456MB
VM Size: Standard_DC2as_v5, Cores: 2, Memory: 8192MB
VM Size: Standard_DC4as_v5, Cores: 4, Memory: 16384MB
VM Size: Standard_DC8as_v5, Cores: 8, Memory: 32768MB
VM Size: Standard_DC16as_v5, Cores: 16, Memory: 65536MB
VM Size: Standard_DC32as_v5, Cores: 32, Memory: 131072MB
VM Size: Standard_DC48as_v5, Cores: 48, Memory: 196608MB
VM Size: Standard_DC64as_v5, Cores: 64, Memory: 262144MB
VM Size: Standard_DC96as_v5, Cores: 96, Memory: 393216MB
VM Size: Standard_DC2ads_v5, Cores: 2, Memory: 8192MB
VM Size: Standard_DC4ads_v5, Cores: 4, Memory: 16384MB
VM Size: Standard_DC

In [13]:
data_asset = ml_client.data.get(name="credit-card-data", version=v2)
credit_df = pd.read_parquet(data_asset.path)

INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.identity._credentials.chained:DefaultAzureCredential acquired a token from AzureCliCredential
INFO:azure.identity._internal.decorators:AzureCliCredential.get_token_info succeeded
INFO:azure.identity._credentials.default:DefaultAzureCredential acquired a token from AzureCliCredential
INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.identity._credentials.chained:DefaultAzureCredential acquired a token from AzureCliCredential
INFO:azure.identity._internal.decorators:AzureCliCredential.get_token_info succeeded
INFO:azure.identity._credentials.default:DefaultAzureCredential acquired a token from AzureCliCredential


In [14]:
credit_df.head()

LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0      20000    2          2         1   24      2      2     -1     -1   
1     120000    2          2         2   26     -1      2      0      0   
2      90000    2          2         2   34      0      0      0      0   
3      50000    2          2         1   37      0      0      0      0   
4      50000    1          2         1   57     -1      0     -1      0   

   PAY_5  ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  \
0     -2  ...          0          0          0         0       689         0   
1      0  ...       3272       3455       3261         0      1000      1000   
2      0  ...      14331      14948      15549      1518      1500      1000   
3      0  ...      28314      28959      29547      2000      2019      1200   
4      0  ...      20940      19146      19131      2000     36681     10000   

   PAY_AMT4  PAY_AMT5  PAY_AMT6  default  
0         0         0         0        1  
1      1000         0      2000        1  
2      1000      1000      5000        0  
3      1100      1069      1000        0  
4      9000       689       679        0  

[5 rows x 24 columns]

In [17]:
credit_df.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default'],
      dtype='object')

In [19]:
df = pd.read_parquet(r"C:\Users\virendra_rathore\Desktop\MLPipeline\data\cleaned-credit-card.parquet")

In [20]:
df.head()

LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0      20000    2          2         1   24      2      2     -1     -1   
1     120000    2          2         2   26     -1      2      0      0   
2      90000    2          2         2   34      0      0      0      0   
3      50000    2          2         1   37      0      0      0      0   
4      50000    1          2         1   57     -1      0     -1      0   

   PAY_5  ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  \
0     -2  ...          0          0          0         0       689         0   
1      0  ...       3272       3455       3261         0      1000      1000   
2      0  ...      14331      14948      15549      1518      1500      1000   
3      0  ...      28314      28959      29547      2000      2019      1200   
4      0  ...      20940      19146      19131      2000     36681     10000   

   PAY_AMT4  PAY_AMT5  PAY_AMT6  default  
0         0         0         0        1  
1      1000         0      2000        1  
2      1000      1000      5000        0  
3      1100      1069      1000        0  
4      9000       689       679        0  

[5 rows x 24 columns]